In [ ]:
from dataclasses import dataclass
from artwork import db, login_manager
from flask_login import UserMixin
from datetime import datetime



@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))


db.metadata.clear()


@dataclass  # dataclass is used to allow for converting objects to JSON in the webservice
class User(db.Model, UserMixin):

    id: int
    username: str
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(20), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    image_file = db.Column(db.String(20), nullable=False, default='default.jpg')
    password = db.Column(db.String(60), nullable=False)

    def __repr__(self):
        return f"<User(id='{self.id}', username='{self.username}', email='{self.email}', image_file='{self.image_file}')>"


class Category(db.Model):
    id = db.Column(db.Integer, primary_key=True, nullable=False)
    name = db.Column(db.String(100), nullable=False)
    description = db.Column(db.TEXT, nullable=True)

    def __repr__(self):
        return f"<Category(id='{self.id}', name='{self.name}', description='{self.description})'>"


class Artwork(db.Model):
    id = db.Column(db.Integer, primary_key=True, nullable=False)
    name = db.Column(db.String(60), nullable=False)
    base = db.Column(db.Integer, nullable=False)
    color = db.Column(db.Integer, nullable=True)
    date_posted = db.Column(db.DateTime, nullable=False)
    size = db.Column(db.Integer, nullable=False)
    frame = db.Column(db.Integer, nullable=False)
    time = db.Column(db.Integer, nullable=False)
    image_file = db.Column(db.String(20), nullable=False, default='default.jpg')

    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    user = db.relationship(User, backref=db.backref('artworks', lazy=True))

    category_id = db.Column(db.Integer, db.ForeignKey('category.id'), nullable=False)
    category = db.relationship(Category, backref=db.backref('artworks', lazy=True))

    def __repr__(self):
        return f"<Artwork(id='{self.id}', name='{self.name}', base='{self.base}', image_file='{self.image_file}," \
               f"color='{self.color}', date_posted='{self.date_posted}', size='{self.size}', frame='{self.frame}', time='{self.time}'," \
               f"user_id='{self.user_id}', category_id='{self.category_id}')>"

    def price(self):
        price = 0
        if self.category.name == 'Abstract':
            a = 1
            while a <= self.size:
                price += a*50
                a += 1

            b = 1
            while b <= self.time:
                price += 200
                b += 1

            c = 1
            while c <= self.color:
                price += 100
                c += 1

            d = 1
            while d <= self.base:
                if d == 1:
                    price -= price*0.2
                elif d == 2:
                    price = price
                else:
                    price += self.size * 20

            if self.frame == 1:
                price = price
            else:
                price += 100

        elif self.category.name == 'Realistic':
            a = 1
            while a <= self.size:
                price += a * 200
                a += 1

            b = 1
            while b <= self.time:
                price += 200
                b += 1

            c = 1
            while c <= self.color:
                price += 100
                c += 1

            d = 1
            while d <= self.base:
                if d == 1:
                    price -= price * 0.2
                elif d == 2:
                    price = price
                else:
                    price += self.size * 20

            if self.frame == 1:
                price = price
            else:
                price += 100

        elif self.category.name == 'Portrait':
            a = 1
            while a <= self.size:
                price += a * 300
                a += 1

            b = 1
            while b <= self.time:
                price += 200
                b += 1

            c = 1
            while c <= self.color:
                price += 100
                c += 1

            d = 1
            while d <= self.base:
                if d == 1:
                    price -= price * 0.2
                elif d == 2:
                    price = price
                else:
                    price += self.size * 20

            if self.frame == 1:
                price = price
            else:
                price += 100

        return price




class Bid(db.Model):
    id = db.Column(db.Integer, primary_key=True, nullable=False)
    bid_price = db.Column(db.Integer, nullable=False)

    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    user = db.relationship(User, backref=db.backref('bids', lazy=True))

    artwork_id = db.Column(db.Integer, db.ForeignKey('artwork.id'), nullable=False)
    artwork = db.relationship(Artwork, backref=db.backref('bids', lazy=True))

    def __repr__(self):
        return f"<Bid(id='{self.id}', bid_price='{self.bid_price}', user_id='{self.user_id}', artwork_id='{self.artwork_id}')>"


In [ ]:
import os
import sys
import random
import datetime
import requests
from artwork import db, bcrypt
from artwork.models import User, Category, Artwork, Bid
from lorem_text import lorem




host = 'localhost'  # host where the system is running
port = 5000  # port where the process is running


def reload_database():
    exit_reload = False
    try:
        response = requests.get(f'http://{host}:{port}')
        print('The website seems to be running. Please stop it and run this file again.', file=sys.stderr)
        exit_reload = True
    except:
        pass
    if exit_reload:
        exit(11)
    try:
        os.remove('artwork/site.db')
        print('previous DB file removed')
    except:
        print('no previous file found')

    db.create_all()

    # creating two users
    hashed_password = bcrypt.generate_password_hash('testing').decode('utf-8')
    default_user1 = User(username='Default',
                         email='default@test.com',
                         image_file='another_pic.jpeg',
                         password=hashed_password)
    db.session.add(default_user1)

    hashed_password = bcrypt.generate_password_hash('testing2').decode('utf-8')
    default_user2 = User(username='Default Second',
                         email='second@test.com',
                         image_file='7798432669b8b3ac.jpg',
                         password=hashed_password)
    db.session.add(default_user2)

    hashed_password = bcrypt.generate_password_hash('testing3').decode('utf-8')
    default_user3 = User(username='Default Third',
                         email='third@test.com',
                         password=hashed_password)
    db.session.add(default_user3)

    # TODO: Here you should include the generation of rows for your database
    cat_1 = Category(name='Abstract')
    cat_2 = Category(name='Realistic')
    cat_3 = Category(name='Portrait')
    categories = [cat_1, cat_2, cat_3]
    db.session.add_all(categories)

    #artworks = []

    for r in range(100):
        user = random.choice([default_user1, default_user2, default_user3])
        name = random.choice(lorem.sentence())  # shouldn't be a sentence but just for testing
        base = random.randint(1, 3)
        color = random.randint(0, 10)
        time = random.randint(1, 100)
        size = random.randint(1, 27)
        frame = random.randint(1, 2)
        date_posted = datetime.datetime.now() - \
                      datetime.timedelta(days=random.randint(0, 160),
                                         hours=random.randint(0, 24),
                                         minutes=random.randint(0, 60))
        category = random.choice(categories)
        artwork = Artwork(user=user,
                          name=name,
                          base=base,
                          color=color,
                          time=time,
                          size=size,
                          frame=frame,
                          date_posted=date_posted,
                          category=category
                          )

        db.session.add(artwork)
        #artworks.append(artworks)

    for b in range(100):
        user = random.choice([default_user1, default_user2, default_user3])
        artwork = (random.choice(Artwork.query.all()))
        bid_price = random.randint(artwork.price(), 50000)
        bid = Bid(bid_price=bid_price, user=user, artwork=artwork)
        db.session.add(bid)

    try:
        db.session.commit()
        print('\nFinalized - database created successfully!')
    except Exception as e:
        print('The operations were not successful. Error:', file=sys.stderr)
        print(e, file=sys.stderr)
        db.session.rollback()


if __name__ == '__main__':
    reload_database()



In [ ]:
print('hello')
